# Querying

This notebook demonstrates Nexus Forge data [querying features](https://nexus-forge.readthedocs.io/en/latest/interaction.html#querying).

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset
from kgforge.core.wrappings.paths import Filter, FilterOperator

## Retrieval

### latest version

In [4]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [5]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [6]:
resource = forge.retrieve(jane.id)

In [7]:
resource == jane

True

### specific version

In [8]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [9]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [10]:
forge.tag(jane, "v1")

<action> _tag_one
<succeeded> True


In [11]:
jane.email = ["jane.doe@epfl.ch", "jane.doe@example.org"]

In [12]:
forge.update(jane)

<action> _update_one
<succeeded> True


In [13]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

3


In [14]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [15]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [16]:
jane_v1_rev = forge.retrieve(jane.id+"?rev=1")

In [17]:
jane_v1 == jane_v1_tag

True

In [18]:
jane_v1 == jane_v1_rev

True

In [19]:
jane_v1 != jane

True

In [20]:
try:
    # DemoStore
    print(jane_v1._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane_v1._store_metadata._rev)

1


### crossbucket retrieval
It is possible to retrieve resources stored in buckets different then the configured one. The configured store should of course support it.

In [21]:
resource = forge.retrieve(jane.id, cross_bucket=True) # cross_bucket defaults to False

In [22]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2023-01-25T13:31:49.657Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/crisely09',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d',
 '_updatedAt': '2023-01-25T13:31:50.140Z',
 '_updatedBy': 'https://

In [23]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [24]:
resource._synchronized

True

### Original source retrieval
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered (retrieve_source=True) without any changes related to store added metadata or JSONLD framing.

In [25]:
resource = forge.retrieve(jane.id, retrieve_source=False) # retrieve_source defaults to True

In [26]:
forge.as_json(resource)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d',
 'type': 'Person',
 'award': 'Nobel',
 'email': ['jane.doe@epfl.ch', 'jane.doe@example.org'],
 'name': 'Jane Doe'}

In [27]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2023-01-25T13:31:49.657Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/crisely09',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/de7fcd9a-7fdc-4097-8c5a-74bc85cb555d',
 '_updatedAt': '2023-01-25T13:31:50.140Z',
 '_updatedBy': 'https://

In [28]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [29]:
resource._synchronized

True

### error handling

In [30]:
resource = forge.retrieve("123")

<action> retrieve
<error> RetrievalError: 404 Client Error: Not Found for url: https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/%3A%2F%2F123/source



In [31]:
resource is None

True

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [32]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [33]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [34]:
dataset = Dataset(forge, type="Dataset", contribution=[contribution_jane, contribution_john])
dataset.add_distribution("../../data/associations.tsv")

In [35]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [36]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/ffb03ef3-5210-4c69-8980-72a5b339b3c3',
 'type': 'Dataset',
 'contribution': [{'type': 'Contribution',
   'agent': {'type': 'Person', 'name': 'Jane Doe'}},
  {'type': 'Contribution', 'agent': {'type': 'Person', 'name': 'John Smith'}}],
 'distribution': {'type': 'DataDownload',
  'atLocation': {'type': 'Location',
   'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault',
    'type': 'DiskStorage',
    '_rev': 1}},
  'contentSize': {'unitCode': 'bytes', 'value': 477},
  'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/crisely09/796d0a36-51b4-429f-9959-abb6dafc474f',
  'digest': {'algorithm': 'SHA-256',
   'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
  'encodingFormat': 'text/tab-separated-values',
  'name': 'associations.tsv'}}

### Using resource paths as filters

The `paths` method load the template or property paths (ie. expected properties) for a given type.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates and types.

In [37]:
p = forge.paths("Dataset")

Autocompletion is enabled on `p` and this can be used to create search filters.

Note: There is a known issue for RdfModel which requires using `p.type.id` instead of `p.type`.

All [python comparison operators](https://www.w3schools.com/python/gloss_python_comparison_operators.asp) are supported.

In [41]:
resources = forge.search(p.type.id=="Person", limit=3)

In [42]:
type(resources)

list

In [43]:
len(resources)

92

In [44]:
forge.as_dataframe(resources)

,id,type,award,name,email,address.type,address.country,address.locality,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,"[jane.doe@epfl.ch, jane.doe@example.org]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,"[jane.doe@epfl.ch, jane.doe@example.org]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,"[jane.doe@epfl.ch, jane.doe@example.org]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,"[jane.doe@epfl.ch, jane.doe@example.org]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,award,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,...,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:18:06.856Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:18:07.458Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:24:17.096Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:24:17.703Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:31:49.101Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T08:55:43.621Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:30:07.586Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:30:08.654Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:31:18.061Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# Search results are not synchronized
resources[0]._synchronized

False

#### Using nested resource property

Property autocompletion is available on a path `p` even for nested properties like `p.contribution`.

In [47]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
resources = forge.search(p.type.id == "Dataset", p.distribution.encodingFormat == "text/tab-separated-values", limit=3)

In [48]:
len(resources)

22

In [49]:
forge.as_dataframe(resources)

,id,type,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name,contribution
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,Interesting Persons,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
3,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
4,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,Interesting Persons,NaN
5,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
6,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
7,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
8,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."
9,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv,NaN,"[{'type': 'Contribution', 'agent': {'type': 'P..."


### Using dictionaries as filters
A dictionary can be provided for filters:
* {'type': {'id':'Dataset'}} is equivalent to p.type.id=="Dataset"
* only the '==' operator is supported
* nested dict are supported
* it is not mandatory for the provided properties and values to be defined in the forge model. Results will be retrieved if there are corresponding data in the store.

This feature is not supported when using the DemoStore


In [50]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
# and created a given dateTime (by default, dateTime values should be signaled by the suffix "^^xsd:dateTime")
filters = {
           "type": "Dataset", 
           "distribution":{"encodingFormat":"text/tab-separated-values"},
           "_createdAt":dataset._store_metadata._createdAt+"^^xsd:dateTime"
          }
resources = forge.search(filters, limit=3)

In [51]:
type(resources)

list

In [52]:
len(resources)

1

In [53]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,...,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2023-01-25T13:31:53.616Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...


### Using built-in Filter objects

#### Supported filter operators

In [54]:
[f"{op.value} ({op.name})" for op in FilterOperator] # These are equivalent to the Python comparison operators

['__eq__ (EQUAL)',
 '__ne__ (NOT_EQUAL)',
 '__lt__ (LOWER_THAN)',
 '__le__ (LOWER_OR_Equal_Than)',
 '__gt__ (GREATER_Than)',
 '__ge__ (GREATER_OR_Equal_Than)']

In [55]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
# and created a given dateTime (dateTime values should be signaled by the suffix "^^xsd:dateTime")
filter_1 = Filter(operator=FilterOperator.EQUAL, path=["type"], value="Dataset")
filter_2 = Filter(operator=FilterOperator.EQUAL, path=["distribution","encodingFormat"], value="text/tab-separated-values")
filter_3 = Filter(operator=FilterOperator.LOWER_OR_Equal_Than, path=["_createdAt"], value=dataset._store_metadata._createdAt+"^^xsd:dateTime")

resources = forge.search(filter_1, filter_2, filter_3, limit=3)

In [56]:
type(resources)

list

In [57]:
len(resources)

22

In [58]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,...,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy,contribution
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-07-20T15:27:08.356Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-07-20T16:53:40.970Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,"[{'type': 'Contribution', 'agent': {'type': 'P..."
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-07-21T08:56:27.014Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,"[{'type': 'Contribution', 'agent': {'type': 'P..."
3,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-07-27T09:12:56.220Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,"[{'type': 'Contribution', 'agent': {'type': 'P..."
4,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-09-27T08:52:13.792Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,NaN
5,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-09-27T08:55:46.766Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,"[{'type': 'Contribution', 'agent': {'type': 'P..."
6,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.

### Using search endpoints

Two types of search endpoints are supported: 'sparql' (default) for graph queries and 'elastic' for document oriented queries. The types of available search endpoint can be configured (see [00-Initialization.ipynb](00%20-%20Initialization.ipynb) for an example of search endpoints config) or set when creating a KnowledgeGraphForge session using the 'searchendpoints' arguments.

The search endpoint to hit when calling forge.search(...) is 'sparql' by default but can be specified using the 'search_endpoint' argument.

#### SPARQL Search Endpoint

In [59]:
# Search for resources of type Person
filters = {"type": "Person"}
resources = forge.search(filters, limit=3, search_endpoint='sparql')

In [60]:
type(resources)

list

In [61]:
len(resources)

92

In [62]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,award,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,...,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:18:06.856Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:18:07.458Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:24:17.096Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:24:17.703Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2023-01-25T13:31:49.101Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T08:55:43.621Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:30:07.586Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:30:08.654Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-09-27T09:31:18.061Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### ElasticSearch Endpoint

In [63]:
# Search for resources of type Person and retrieve their ids and names.
filters = {"@type": "http://schema.org/Person"}
resources = forge.search(filters, limit=3, 
                         search_endpoint='elastic', 
                         includes=["@id", "@type"]) # fields can also be excluded with 'excludes'

In [64]:
type(resources)

list

In [65]:
len(resources)

3

In [66]:
forge.as_dataframe(resources, store_metadata=True)

,id,type
0,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person
1,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person
2,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person


In [67]:
# Search results are not synchronized
resources[0]._synchronized

False

In [68]:
resources[0].id

'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/0b5441a9-1db9-4cdb-8ec0-afbc4bd25bb5'

In [69]:
resources[0].type

'http://schema.org/Person'

### Crossbucket search
It is possible to search for resources stored in buckets different than the configured one. The configured store should of course support it.

In [70]:
resources = forge.search(p.type.id == "Association", limit=3, cross_bucket=True)  # cross_bucket defaults to False

In [71]:
type(resources)

list

In [72]:
len(resources)

28

In [73]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,...,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name,distribution,agent.id,agent
0,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': 'DataDownload', 'atLocation': {'type...",NaN,NaN
7,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://sandbox.bluebrainnexus.io/v1/resources...,NaN
8,https://sandbox.bluebrainnexus.io/v1/resources...,Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'id': 'https://sandbox.bluebrainnexus.io/v1/...
9,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association,NaN,NaN,NaN


In [74]:
#Furthermore it is possible to filter by bucket when cross_bucket is set to True. Setting a bucket value when cross_bucket is False will trigger a not_supported exception.
resources = forge.search(p.type.id == "Person", limit=3, cross_bucket=True, bucket="dke/kgforge")  # add a bucket

In [75]:
type(resources)

list

In [76]:
len(resources)

0

In [77]:
forge.as_dataframe(resources)

""


### Searching original source
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered (retrieve_source=True) without any changes related to store added metadata or JSONLD framing.

In [78]:
resources = forge.search(p.type.id == "Association", limit=3, retrieve_source=False)  # retrieve_source defaults to True

In [79]:
type(resources)

list

In [80]:
len(resources)

28

In [81]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,...,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name,distribution,agent.id,agent
0,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': 'DataDownload', 'atLocation': {'type...",NaN,NaN
7,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://sandbox.bluebrainnexus.io/v1/resources...,NaN
8,https://sandbox.bluebrainnexus.io/v1/resources...,Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'id': 'https://sandbox.bluebrainnexus.io/v1/...
9,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,...,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association,NaN,NaN,NaN


## Graph traversing

SPARQL is used as a query language to perform graph traversing.

Nexus Forge implements a SPARQL query rewriting strategy leveraging a configured RDFModel that lets users write SPARQL queries without adding prefix declarations, prefix names or long IRIs. With this strategy, only type and property names can be provided.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel.

In [82]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [83]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [84]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [85]:
forge.register(association)

<action> _register_one
<succeeded> True


In [86]:
forge.template("Dataset") # Templates help know which property to use when writing a query to serach for a given type

{
    id: ""
    type:
    {
        id: ""
    }
    annotation:
    {
        id: ""
        type: Annotation
        hasBody:
        {
            id: ""
            type:
            {
                id: ""
            }
            label: ""
            note: ""
        }
        hasTarget:
        {
            id: ""
            type: AnnotationTarget
        }
        note: ""
    }
    brainLocation:
    {
        id: ""
        type: BrainLocation
        atlasSpatialReferenceSystem:
        {
            id: ""
            type: AtlasSpatialReferenceSystem
        }
        brainRegion:
        {
            id: ""
            label: ""
        }
        coordinatesInBrainAtlas:
        {
            id: ""
            valueX: 0.0
            valueY: 0.0
            valueZ: 0.0
        }
        coordinatesInSlice:
        {
            spatialReferenceSystem:
            {
                id: ""
                type: SpatialReferenceSystem
            }
            valueX

### Prefix and namespace free SPARQL query

When a forge RDFModel is configured, then there is no need to provide prefixes and namespaces when writing a SPARQL query. Prefixes and namespaces will be automatically inferred from the provided schemas and/or JSON-LD context and the query rewritten accordingly.

In [87]:
query = """
    SELECT ?id ?name ?contributor
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [88]:
resources = forge.sparql(query, limit=3)

In [89]:
type(resources)

list

In [90]:
len(resources)

82

In [91]:
print(resources[0])

{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/c2f62d9e-d1ea-4baa-92e2-2bb96d9f9d6d
    contributor: t3914
    name: Jane Doe
}


In [92]:
forge.as_dataframe(resources)

,id,contributor,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,t3914,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,t3918,Jane Doe
2,https://sandbox.bluebrainnexus.io/v1/resources...,t3920,John Smith
3,https://sandbox.bluebrainnexus.io/v1/resources...,t4013,Jane Doe
4,https://sandbox.bluebrainnexus.io/v1/resources...,t4015,John Smith
...,...,...,...
77,https://sandbox.bluebrainnexus.io/v1/resources...,t3880,Jane Doe
78,https://sandbox.bluebrainnexus.io/v1/resources...,t3882,John Smith
79,https://sandbox.bluebrainnexus.io/v1/resources...,t3901,John Smith
80,https://sandbox.bluebrainnexus.io/v1/resources...,t3903,Jane Doe


### display rewritten SPARQL query 

In [93]:
resources = forge.sparql(query, limit=3, debug=True)

Submitted query:
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   
       SELECT ?id ?name ?contributor
       WHERE {
           ?id a schema:Dataset ;
           nsg:contribution/prov:agent ?contributor.
           ?c

### Full SPARQL query

Regular SPARQL query can also be provided. When provided, the limit and offset arguments superseed any in query limit or offset values.

In [94]:
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
   ORDER BY ?id
   LIMIT 1
   OFFSET 0
"""

In [95]:
# it is recommended to set 'rewrite' to 'False' to prevent the sparql query rewriting when a syntactically correct SPARQL query is provided.
resources = forge.sparql(query, rewrite=False, limit=3, offset=1, debug=True) 

Submitted query:
   
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
      PREFIX dcat: <http://www.w3.org/ns/dcat#>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
      PREFIX nsg: <https://neuroshapes.org/>
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX prov: <http://www.w3.org/ns/prov#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX schema: <http://schema.org/>
      PREFIX sh: <http://www.w3.org/ns/shacl#>
      PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX vann: <http://purl.org/vocab/vann/>
      PREFIX void: <http://rdfs.org/ns/void#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      PREFIX : <https://neuroshapes.org/>
      SELECT ?id ?name
      WHERE {
          ?id a schema:Dataset ;
          nsg:contribution/prov:ag

In [96]:
type(resources)

list

In [97]:
len(resources)

3

In [98]:
type(resources[0])

kgforge.core.resource.Resource

In [99]:
forge.as_dataframe(resources)

,id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
2,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith


### Avoid rewriting the query

To not assign any limit or offset, one can pass `None` to those parameters

In [100]:
query_without_limit = """
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
   ORDER BY ?id
"""

In [101]:

resources = forge.sparql(query_without_limit, rewrite=False, limit=None, offset=None, debug=True)

Submitted query:
   
      PREFIX dc: <http://purl.org/dc/elements/1.1/>
      PREFIX dcat: <http://www.w3.org/ns/dcat#>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
      PREFIX nsg: <https://neuroshapes.org/>
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX prov: <http://www.w3.org/ns/prov#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX schema: <http://schema.org/>
      PREFIX sh: <http://www.w3.org/ns/shacl#>
      PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX vann: <http://purl.org/vocab/vann/>
      PREFIX void: <http://rdfs.org/ns/void#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      PREFIX : <https://neuroshapes.org/>
      SELECT ?id ?name
      WHERE {
          ?id a schema:Dataset ;
          nsg:contribution/prov

In [102]:
len(resources)

84

If you only want to add the context, but keep the content of the query the same, you need to set the `rewrite` parameter to `False`

In [103]:
query_without_context = """
    SELECT ?id ?name ?contributor
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [104]:
resources = forge.sparql(query_without_context, limit=None, debug=True)

Submitted query:
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   
       SELECT ?id ?name ?contributor
       WHERE {
           ?id a schema:Dataset ;
           nsg:contribution/prov:agent ?contributor.
           ?c

In [105]:
len(resources)

84

## ElasticSearch DSL Query

ElasticSearch DSL can be used as a query language search for resources provided that the configured store supports it. The 'BlueBrainNexusStore' supports ElasticSearch.

Note: DemoStore doesn't implement ElasaticSearch DSL operations.

In [106]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [107]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [108]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [109]:
forge.register(association)

<action> _register_one
<succeeded> True


### Plain ElasticSearch DSL

In [110]:
query = """
        {
          "_source": {
            "includes": [
              "@id",
              "name"
            ]
          },
          "query": {
            "term": {
              "@type": "http://schema.org/Dataset"
            }
          }
        }
"""

In [111]:
# limit and offset (when provided in this method call) superseed 'size' and 'from' values provided in the query
resources = forge.elastic(query, limit=3)

In [112]:
type(resources)

list

In [113]:
len(resources)

3

In [114]:
type(resources[0])

kgforge.core.resource.Resource

In [115]:
forge.as_dataframe(resources)

,id,name
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,Cux2-CreERT2;Ai14-205530.03.02.01
1,https://bbp.epfl.ch/neurosciencegraph/data/neu...,Cux2-CreERT2;Ai14-205530.03.02.01
2,https://bbp.epfl.ch/neurosciencegraph/data/neu...,Cux2-CreERT2;Ai14-205530.03.02.01


## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [116]:
jane = Resource(type="Person", name="Jane Doe")

In [117]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons-with-id.csv
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [118]:
distribution = forge.attach("../../data")

In [119]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [120]:
forge.register(association)

<action> _register_one
<succeeded> True


In [ ]:
# By default, the downladable file urls are collected from the json path "distribution.contentUrl" (follow="distribution.contentUrl") and
# the files are downloaded in the current path (path="."). 
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.
# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
forge.download(association)

In [ ]:
# Specific content type can be downloaded.
forge.download(association, content_type="text/tab-separated-values")

In [195]:
# The urls or the files to download can be collected from a different json path (by setting a value for "follow") and 
# the files downloaded to a different path (by setting a value for "path")
forge.download(association, follow="distribution.contentUrl", path="./downloaded/")

In [122]:
! ls -l ./downloaded/

total 1344
-rw-r--r--  1 cgonzale  10067     477 Jan 18 13:17 associations.tsv
-rw-r--r--  1 cgonzale  10067     477 Jan 25 14:24 associations.tsv.20230125142439
-rw-r--r--  1 cgonzale  10067     477 Jan 25 14:34 associations.tsv.20230125143400
-rw-r--r--  1 cgonzale  10067      16 Jan 18 13:17 my_data.xwz
-rw-r--r--  1 cgonzale  10067      16 Jan 25 14:24 my_data.xwz.20230125142439
-rw-r--r--  1 cgonzale  10067      16 Jan 25 14:34 my_data.xwz.20230125143400
-rw-r--r--  1 cgonzale  10067      24 Jan 18 13:17 my_data_derived.txt
-rw-r--r--  1 cgonzale  10067      24 Jan 25 14:24 my_data_derived.txt.20230125142439
-rw-r--r--  1 cgonzale  10067      24 Jan 25 14:34 my_data_derived.txt.20230125143400
-rw-r--r--  1 cgonzale  10067     126 Jan 18 13:17 persons-with-id.csv
-rw-r--r--  1 cgonzale  10067     126 Jan 25 14:24 persons-with-id.csv.20230125142439
-rw-r--r--  1 cgonzale  10067     126 Jan 25 14:34 persons-with-id.csv.20230125143400
-rw-r--r--  1 cgonzale  10067      52 Jan 18 13:17

In [123]:
#! rm -R ./downloaded/